In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 

# Librerías para el procesamiento de access logs 
import re 
from parse import parse 
from lars.apache import ApacheSource, COMBINED, ApacheWarning

# Manejo de advertencias del sistema, usada para capturar las líneas que no pueden parsearse por problemas de lars (ApacheWarning)
import warnings

# Configuración de estilo para las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Ejecutar antes las celdas de `01_access_log_parsing` o elige una ubicación donde exista un archivo de access logs (Apache Combined Log Format) 
log_file_path = '../data/target/access_log_master.log'
log_prep_path = '../data/target/access_log_master.csv'

### Expresión General para Apache Combined Log Format

In [ ]:
LOG_PATTERN = re.compile(
  r'(?P<ip_client>\S+)\s+'      # IP del cliente 
  r'(?P<ident>\S+)\s+'          # ident 
  r'(?P<auth_user>\S+)\s+'      # auth_user
  r'\[(?P<timestamp>.+?)\]\s+'  # timestamp
  r'\"(?P<request>.*?)\"\s+'    # línea completa de request: método + url + protocolo
  r'(?P<status>\d{3})\s+'       # código de estado
  r'(?P<size>\S+)\s+'           # tamaño de respuesta (bytes)
  r'\"(?P<referer>.*?)\"\s+'    # referer
  r'\"(?P<user_agent>.*?)\"'    # user-agent
)

def parse_apache_logs(log_file_path, add_n_line:bool=False, del_request:bool=True):
  "Parsea logs de Apache extrayendo todos los campos solicitados"
  parsed_logs = []
  failed_lines = []
  
  with open(log_file_path, 'r', encoding='utf-8', errors='ignore') as file:
    for line_num, line in enumerate(file, 1):
      line = line.strip()
      if not line:
        continue
      
      match = LOG_PATTERN.match(line)
      
      if match:
        # Obtener todos los campos capturados
        log_data = match.groupdict()
        
        # Separar request en método, URL y protocolo
        request_parts = log_data['request'].split()
        if len(request_parts) >= 3:
          log_data['method'] = request_parts[0]
          log_data['url'] = request_parts[1]
          log_data['protocol'] = request_parts[2]
        elif len(request_parts) == 2:
          log_data['method'] = request_parts[0]
          log_data['url'] = request_parts[1]
          log_data['protocol'] = 'UNKNOWN'
        else:
          log_data['method'] = 'UNKNOWN'
          log_data['url'] = 'UNKNOWN'
          log_data['protocol'] = 'UNKNOWN'
        
        if add_n_line:
          # Añadir número de línea
          log_data['line_number'] = line_num
        
        if del_request:
          del log_data['request']
        
        parsed_logs.append(log_data)
      else:
        # Guardar línea fallida
        failed_lines.append({
          'line': line_num,
          'content': line[:100]
        })
  
  return parsed_logs, failed_lines

logs, errors = parse_apache_logs(log_file_path)

print(f"Cantidad de logs parseados: {len(logs)}")
print(f"Cantidad de logs fallidos:  {len(errors)}")

In [ ]:
df = pd.DataFrame(logs)

# Columna para representar si un log es un ataque o no (inicialmente todos en False)
df['attack'] = False 

display(df.head())
display(df.tail())

In [ ]:
#df.to_csv(log_prep_path, index=False)

In [ ]:
# transformar 'timestamp' a datetime (ajustando el formato de Apache) 
def parse_apache_time(time_str):
  "Convierte el formato de tiempo de Apache Access Log a datetime"
  try:
    # Ejemplo de formato: [01/01/2001:00:00:01 -0700]
    return pd.to_datetime(time_str, format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
  except: 
    return pd.NaT 

df['timestamp'] = df['timestamp'].apply(parse_apache_time)


# transformar 'size' a numérico (manejando valores no numéricos como '-')
df['size'] = pd.to_numeric(df['size'], errors='coerce').fillna(0)

In [ ]:
df['timestamp']

In [ ]:
print(f"Rango de Tiempo: {df['timestamp'].min()} -> {df['timestamp'].max()}")
print(f"Estadísticas de 'size': Min={ df['size'].min() }, Max={ df['size'].max() }, Mean={ df['size'].mean() }")

In [ ]:
# Mostrar las primeras K filas con los valores 'size' más altos
K = 10
display_cols = [
  
]
# Ordenar por tamaño de forma descendente y tomar las primeras K filas
top_K_largest = df.nlargest(K, 'size')
above_mean = df[df['size'] > df['size'].mean()]
print(f"Logs con tamaño de respuesta > promedio (promedio/mean={df['size'].mean():.2f}): {len(above_mean)} ({(len(above_mean)/len(df))*100:.2f}%)")
display(top_K_largest)

In [ ]:
plt.figure(figsize=(15, 6))

# Crear scatter plot con transparencia para manejar muchos puntos
plt.scatter(df['timestamp'], df['size'], alpha=0.6, s=10, color='royalblue')

plt.title('Tamaño de respuestas a lo largo del tiempo', fontsize=14, fontweight='bold')
plt.xlabel('Fecha y Hora', fontsize=12)
plt.ylabel('Tamaño (bytes)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()